In [ ]:
import pandas as pd
from ta.trend import SMAIndicator, MACD
from ta.momentum import RSIIndicator
import pynance as pn
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
#loading the datas
file_paths = {
    'AAPL': '../data/AAPL_historical_data.csv',
    'GOOGL': '../data/AMZN_historical_data.csv',
    'AMZN': '../data/GOOG_historical_data.csv',
    'MSFT': '../data/META_historical_data.csv',
    'TSLA': '../data/MSFT_historical_data.csv',
    'META': '../data/NVDA_historical_data.csv',
    'NFLX': '../data/TSLA_historical_data.csv'
}

stock_data = {}

for company, path in file_paths.items():
    df = pd.read_csv(path, parse_dates=['Date'])
    df.set_index('Date', inplace=True) 
    stock_data[company] = df


In [3]:
#cleaning - drop any missing datasets
for name, df in stock_data.items():
    stock_data[name] = df.dropna()
    print(f"{name}: {stock_data[name].isnull().sum().sum()} missing values after dropping")


AAPL: 0 missing values after dropping
GOOGL: 0 missing values after dropping
AMZN: 0 missing values after dropping
MSFT: 0 missing values after dropping
TSLA: 0 missing values after dropping
META: 0 missing values after dropping
NFLX: 0 missing values after dropping


In [4]:
#technical indicators
def apply_technical_indicators(df, window_sma=20, window_rsi=14):
    df = df.copy()
    
    # SMA
    sma = SMAIndicator(close=df['Close'], window=window_sma)
    df['SMA'] = sma.sma_indicator()
    
    # RSI
    rsi = RSIIndicator(close=df['Close'], window=window_rsi)
    df['RSI'] = rsi.rsi()
    
    # MACD
    macd = MACD(close=df['Close'])
    df['MACD'] = macd.macd()
    df['MACD_signal'] = macd.macd_signal()
    df['MACD_diff'] = macd.macd_diff()
    
    return df.dropna()

In [5]:
#apply for all companies
for company in stock_data:
    stock_data[company] = apply_technical_indicators(stock_data[company])

In [7]:
tickers = ['AAPL', 'GOOGL', 'AMZN', 'MSFT', 'TSLA', 'META', 'NFLX']
stock_metrics = {}

for ticker in tickers:
    df = pn.data.get(ticker, start='2022-01-01', end='2023-01-01')
    daily_returns = df['Close'].pct_change()
    stock_metrics[ticker] = {
        'data': df,
        'daily_returns': daily_returns,
        'cumulative_returns': (1 + daily_returns).cumprod(),
        'volatility': daily_returns.std(),
        'rolling_mean': df['Close'].rolling(20).mean(),
        'rolling_std': df['Close'].rolling(20).std()
    }
